### Libraries & Packages

In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
import sklearn
import matplotlib.pyplot as plt

### Loading Data

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

df = pd.concat([train, test], axis = 0)

/var/folders/kz/3t7pyllj7w3fy7h055vlb0p00000gn/T/ipykernel_11399/4238871333.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/train.csv")


In [3]:
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.906667,10,...,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343,NaN
49996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066,NaN
49997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098,NaN
49998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717,NaN


# Sorting and Cleaning Training Data

#### Removing Duplicates (by Customer_ID)

In [4]:
df = df.drop_duplicates(subset = 'Customer_ID', keep = 'first')

In [5]:
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
16,0x161a,CUS_0x2dbc,January,Langep,34,486-85-3974,_______,143162.64,12187.220000,1,...,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
24,0x1626,CUS_0xb891,January,Jasond,54,072-31-6145,Entrepreneur,30689.89,2612.490833,2,...,Good,632.46,26.544229,17 Years and 3 Months,No,16.415452,81.22885871073616,Low_spent_Large_value_payments,433.6047729627723,Standard
32,0x1632,CUS_0x1cdb,January,Deepaa,21,615-06-7821,Developer,35547.71_,2853.309167,7,...,Standard,943.86,39.797764,30 Years and 8 Months,Yes,0.000000,276.72539431736266,!@9#%8,288.60552234930395,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99960,0x25fb6,CUS_0x372c,January,Lucia Mutikanik,18,340-85-7301,Lawyer,42903.79,3468.315833,0,...,Good,1079.48,27.289440,28 Years and 1 Months,No,50894.000000,78.51494451125625,High_spent_Small_value_payments,493.341182,Good
99968,0x25fc2,CUS_0xf16,January,Maria Sheahanb,44,#F%$D@*&8,Media_Manager,16680.35,1528.029167,1,...,Good,897.16,39.868572,NaN,NM,41.113561,52.95197781627658,High_spent_Small_value_payments,318.737378,Good
99976,0x25fce,CUS_0xaf61,January,Chris Wickhamm,49,133-16-7738,Writer,37188.1,3097.008333,1,...,Good,620.64,39.080823,29 Years and 9 Months,No,84.205949,223.87501818278344,Low_spent_Small_value_payments,291.619866,Good
99984,0x25fda,CUS_0x8600,January,Sarah McBridec,28,031-35-0942,Architect,20002.88,1929.906667,10,...,Bad,3571.7_,22.895966,5 Years and 8 Months,Yes,60.964772,43.37067007268804,High_spent_Large_value_payments,328.655224,Poor


#### Removing Underscores

In [6]:
# Define a function to check for underscores next to numbers
def contains_underscore_next_to_number(series):
    series = series.astype(str)
    return series.str.contains(r'\d_') | series.str.contains(r'_')

# Apply the function to the relevant columns
columns_to_check = ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
                     'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
                     'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries',
                     'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
                     'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 
                     'Monthly_Balance'] 

# Boolean mask for rows to keep (those without underscores next to numbers)
mask = ~df[columns_to_check].apply(contains_underscore_next_to_number).any(axis=1)

# Filter the DataFrame to keep only the rows that do not contain underscores next to numbers
df_filtered = df[mask]

df_filtered

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
16,0x161a,CUS_0x2dbc,January,Langep,34,486-85-3974,_______,143162.64,12187.220000,1,...,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
24,0x1626,CUS_0xb891,January,Jasond,54,072-31-6145,Entrepreneur,30689.89,2612.490833,2,...,Good,632.46,26.544229,17 Years and 3 Months,No,16.415452,81.22885871073616,Low_spent_Large_value_payments,433.6047729627723,Standard
40,0x163e,CUS_0x95ee,January,Np,31,612-70-8987,Lawyer,73928.46,NaN,4,...,Good,548.2,39.962685,NaN,No,15015.000000,98.67440994166124,High_spent_Large_value_payments,740.1960900583389,Good
48,0x164a,CUS_0x284a,January,Nadiaq,33,411-51-0676,Lawyer,131313.4,11242.783333,0,...,Good,352.16,32.200509,30 Years and 7 Months,NM,137.644605,378.1712535207537,High_spent_Medium_value_payments,858.462474411158,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99928,0x25f86,CUS_0xb11c,January,Yinka Adegokej,38,546-94-4789,Manager,15319.65,NaN,6,...,Bad,1453.61,36.518222,10 Years and 9 Months,Yes,28.182033,167.88843516537085,Low_spent_Small_value_payments,239.993281,Poor
99936,0x25f92,CUS_0xad4f,January,Sabina Zawadzkig,47,226-45-0652,Developer,22620.79,1722.065833,7,...,Standard,642.46,31.605703,29 Years and 3 Months,No,0.000000,56.072675001956775,High_spent_Medium_value_payments,366.133908,Standard
99960,0x25fb6,CUS_0x372c,January,Lucia Mutikanik,18,340-85-7301,Lawyer,42903.79,3468.315833,0,...,Good,1079.48,27.289440,28 Years and 1 Months,No,50894.000000,78.51494451125625,High_spent_Small_value_payments,493.341182,Good
99968,0x25fc2,CUS_0xf16,January,Maria Sheahanb,44,#F%$D@*&8,Media_Manager,16680.35,1528.029167,1,...,Good,897.16,39.868572,NaN,NM,41.113561,52.95197781627658,High_spent_Small_value_payments,318.737378,Good


#### Removing ID, Customer ID, Month, Name, Age, SSN, Occupation

In [7]:
columns_to_remove = ['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Type_of_Loan']

df_filtered = df_filtered.drop(columns=columns_to_remove, axis = 1)
df_filtered

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
8,34847.84,3037.986667,2,4,6,1,3,4,5.42,2.0,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
16,143162.64,12187.220000,1,5,8,3,5,8,7.1,3.0,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
24,30689.89,2612.490833,2,5,4,1,0,6,1.99,4.0,Good,632.46,26.544229,17 Years and 3 Months,No,16.415452,81.22885871073616,Low_spent_Large_value_payments,433.6047729627723,Standard
40,73928.46,NaN,4,1288,8,0,12,10,10.14,2.0,Good,548.2,39.962685,NaN,No,15015.000000,98.67440994166124,High_spent_Large_value_payments,740.1960900583389,Good
48,131313.4,11242.783333,0,1,8,2,0,3,9.34,2.0,Good,352.16,32.200509,30 Years and 7 Months,NM,137.644605,378.1712535207537,High_spent_Medium_value_payments,858.462474411158,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99928,15319.65,NaN,6,7,15,4,51,18,17.13,6.0,Bad,1453.61,36.518222,10 Years and 9 Months,Yes,28.182033,167.88843516537085,Low_spent_Small_value_payments,239.993281,Poor
99936,22620.79,1722.065833,7,3,9,0,27,18,5.31,2.0,Standard,642.46,31.605703,29 Years and 3 Months,No,0.000000,56.072675001956775,High_spent_Medium_value_payments,366.133908,Standard
99960,42903.79,3468.315833,0,4,6,1,14,0,4.1,1.0,Good,1079.48,27.289440,28 Years and 1 Months,No,50894.000000,78.51494451125625,High_spent_Small_value_payments,493.341182,Good
99968,16680.35,1528.029167,1,1,5,4,4,0,5.69,4.0,Good,897.16,39.868572,NaN,NM,41.113561,52.95197781627658,High_spent_Small_value_payments,318.737378,Good


#### Dropping NA Values

In [8]:
print(df_filtered.shape)
df_filtered = df_filtered.dropna()
print(df_filtered.shape)

df_filtered


(8018, 20)
(5332, 20)


,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
8,34847.84,3037.986667,2,4,6,1,3,4,5.42,2.0,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
16,143162.64,12187.220000,1,5,8,3,5,8,7.1,3.0,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
24,30689.89,2612.490833,2,5,4,1,0,6,1.99,4.0,Good,632.46,26.544229,17 Years and 3 Months,No,16.415452,81.22885871073616,Low_spent_Large_value_payments,433.6047729627723,Standard
48,131313.4,11242.783333,0,1,8,2,0,3,9.34,2.0,Good,352.16,32.200509,30 Years and 7 Months,NM,137.644605,378.1712535207537,High_spent_Medium_value_payments,858.462474411158,Good
64,114838.41,9843.867500,2,5,7,-100,13,11,8.24,3.0,Good,1377.74,33.664554,21 Years and 4 Months,No,226.892792,215.19351594560425,High_spent_Small_value_payments,802.3004421328528,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99864,33702.74,2998.561667,7,3,19,2,6,8,10.7,4.0,Standard,636.96,25.382445,23 Years and 11 Months,Yes,37.993229,94.72366141883847,High_spent_Medium_value_payments,417.139276,Standard
99912,20059.98,1523.665000,8,5,12,4,25,11,2.97,3.0,Good,909.01,30.568185,16 Years and 3 Months,No,6980.000000,151.21408617712285,Low_spent_Medium_value_payments,236.075587,Poor
99936,22620.79,1722.065833,7,3,9,0,27,18,5.31,2.0,Standard,642.46,31.605703,29 Years and 3 Months,No,0.000000,56.072675001956775,High_spent_Medium_value_payments,366.133908,Standard
99960,42903.79,3468.315833,0,4,6,1,14,0,4.1,1.0,Good,1079.48,27.289440,28 Years and 1 Months,No,50894.000000,78.51494451125625,High_spent_Small_value_payments,493.341182,Good


#### One-Hot Encoding of Categorical Column & X_train and Y_train

In [9]:
df_filtered = df_filtered.drop(df_filtered[df_filtered['Payment_Behaviour'] == "!@9#%8"].index)

X_train = df_filtered.iloc[:, 0:19]
Y_train = df_filtered['Credit_Score']

X_train = pd.get_dummies(X_train, columns = ['Payment_Behaviour', 'Credit_Mix', 'Payment_of_Min_Amount'], 
                             dtype=int)


In [10]:
print(f'Shape of X_train: {X_train.shape} \nShape of Y_train: {Y_train.shape}')

Shape of X_train: (4959, 28) 
Shape of Y_train: (4959,)


In [11]:
X_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,...,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes
8,34847.84,3037.986667,2,4,6,1,3,4,5.42,2.0,...,0,0,0,1,0,1,0,0,1,0
24,30689.89,2612.490833,2,5,4,1,0,6,1.99,4.0,...,0,1,0,0,0,1,0,0,1,0
48,131313.4,11242.783333,0,1,8,2,0,3,9.34,2.0,...,0,0,0,0,0,1,0,1,0,0
64,114838.41,9843.867500,2,5,7,-100,13,11,8.24,3.0,...,1,0,0,0,0,1,0,0,1,0
88,88640.24,7266.686667,3,6,1,2,-1,0,3.51,3.0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99864,33702.74,2998.561667,7,3,19,2,6,8,10.7,4.0,...,0,0,0,0,0,0,1,0,0,1
99912,20059.98,1523.665000,8,5,12,4,25,11,2.97,3.0,...,0,0,1,0,0,1,0,0,1,0
99936,22620.79,1722.065833,7,3,9,0,27,18,5.31,2.0,...,0,0,0,0,0,0,1,0,1,0
99960,42903.79,3468.315833,0,4,6,1,14,0,4.1,1.0,...,1,0,0,0,0,1,0,0,1,0


In [12]:
Y_train

8        Standard
24       Standard
48           Good
64           Good
88       Standard
           ...   
99864    Standard
99912        Poor
99936    Standard
99960        Good
99976        Good
Name: Credit_Score, Length: 4959, dtype: object

#### Credit History (Age)

In [13]:
from decimal import Decimal

#Map function to turn string sequence into double
def convert_to_age_double(str):
    if str[1] == " ":
        years = str[0]
        if str[13] == " ":
            months = str[12]
        else:
            months = str[12:14]
    else :
        years = str[0:2]
        if str[14] == " ":
            months = str[13]
        else:
            months = str[13:15]

    return round(Decimal(years) + (Decimal(months) / 12), 2)

X_train['Credit_History_Age'] = X_train['Credit_History_Age'].map(convert_to_age_double)

#### Ensure Values are Numerical

In [14]:
columns_to_convert = ['Annual_Income', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 
                      'Changed_Credit_Limit', 'Outstanding_Debt', 'Credit_History_Age', 'Amount_invested_monthly',
                      'Monthly_Balance']

# Function to convert columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')

convert_columns_to_numeric(X_train, columns_to_convert)


In [15]:
X_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,...,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes
8,34847.84,3037.986667,2,4,6,1,3,4,5.42,2.0,...,0,0,0,1,0,1,0,0,1,0
24,30689.89,2612.490833,2,5,4,1,0,6,1.99,4.0,...,0,1,0,0,0,1,0,0,1,0
48,131313.40,11242.783333,0,1,8,2,0,3,9.34,2.0,...,0,0,0,0,0,1,0,1,0,0
64,114838.41,9843.867500,2,5,7,-100,13,11,8.24,3.0,...,1,0,0,0,0,1,0,0,1,0
88,88640.24,7266.686667,3,6,1,2,-1,0,3.51,3.0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99864,33702.74,2998.561667,7,3,19,2,6,8,10.70,4.0,...,0,0,0,0,0,0,1,0,0,1
99912,20059.98,1523.665000,8,5,12,4,25,11,2.97,3.0,...,0,0,1,0,0,1,0,0,1,0
99936,22620.79,1722.065833,7,3,9,0,27,18,5.31,2.0,...,0,0,0,0,0,0,1,0,1,0
99960,42903.79,3468.315833,0,4,6,1,14,0,4.10,1.0,...,1,0,0,0,0,1,0,0,1,0


#### Standardization

In [16]:
def displayHist(data, threshold, bins, func, name, fig):
    data = data.to_numpy()
    dist = data[data <= threshold]
    if func: 
        dist = func(data)
    plt.figure(fig)
    plt.hist(dist.tolist(), bins)
    plt.title(name)

epsilon = 1e-5

# displayHist(X_train['Annual_Income'],  1e5, 30, np.log, 'Annual Income', 1)
# displayHist(X_train['Monthly_Inhand_Salary'],  1e5, 12, np.log, 'Monthly Salary', 2)
# displayHist(X_train['Outstanding_Debt'],  1e5, 15, np.cbrt, 'Outstanding_Debt', 3)
# displayHist(X_train['Credit_Utilization_Ratio'], 1e5, 20, None, 'Credit Utilization Ratio', 4)
# displayHist(X_train['Credit_History_Age'], 1e2, 20, None, 'Credit_History_Age Ratio', 5)
# displayHist(X_train['Total_EMI_per_month'] + epsilon, 1e4, 20, np.log, 'Total_EMI_per_month', 6)
# displayHist(X_train['Amount_invested_monthly'] + epsilon, 1e5, 20, np.log, 'Amount_invested_monthly', 7)
# displayHist(X_train['Monthly_Balance'] + epsilon, 1e5, 20, np.log, 'Monthly_Balance', 8)
# displayHist(X_train['Num_Bank_Accounts'], 20, 5, None, 'Num_Bank_Accounts', 9)
# displayHist(X_train['Num_Credit_Card'], 15, 7, None, 'Num_Credit_Card', 10)
# displayHist(X_train['Interest_Rate'], 1e2, 20, np.log, 'Interest_Rate', 11)
# displayHist(X_train['Delay_from_due_date'], 50, 10, None, 'Delay_from_due_date', 12)
# displayHist(X_train['Num_of_Delayed_Payment'], 50, 10, None, 'Num_of_Delayed_Payment', 13)
# displayHist(X_train['Num_Credit_Inquiries'] + epsilon, 50, 10, np.log, 'Num_Credit_Inquiries', 14)
# displayHist(X_train['Num_of_Loan'] + epsilon, 30, 10, np.log, 'Num_of_Loan', 15)
# displayHist(X_train['Changed_Credit_Limit'] + epsilon, 50, 10, np.log, 'Changed_Credit_Limit', 16)


#### X_Train (Gaussian - Log)

In [17]:
def transformation(df, func, cols):
    for name in cols:
        df[name] = func(df[name] + epsilon)
        print(f"{name} is transformed")

columns_to_log = ['Annual_Income', 'Monthly_Inhand_Salary', 'Total_EMI_per_month', 'Amount_invested_monthly', 
                  'Monthly_Balance', 'Interest_Rate', 'Num_Credit_Inquiries', 'Num_of_Loan', 'Changed_Credit_Limit']

transformation(X_train, np.log, columns_to_log)

Annual_Income is transformed
Monthly_Inhand_Salary is transformed
Total_EMI_per_month is transformed
Amount_invested_monthly is transformed
Monthly_Balance is transformed
Interest_Rate is transformed
Num_Credit_Inquiries is transformed
Num_of_Loan is transformed
Changed_Credit_Limit is transformed


/opt/anaconda3/envs/default/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


#### X_Train (Gaussian - Cube Root)

In [18]:
# transformation(X_train, np.cbrt, 'Outstanding_Debt')

X_train['Outstanding_Debt'] = np.cbrt(X_train['Outstanding_Debt'])

#### Standardizing X_Train

In [19]:
from sklearn.preprocessing import StandardScaler

scaleStandard = StandardScaler()

standardize_cols = X_train.iloc[:, 0:16]

encoded_cols = X_train.iloc[:, 16:]

standardize_cols = scaleStandard.fit_transform(standardize_cols)

In [20]:
standardize_cols = pd.DataFrame(standardize_cols, 
                                columns = ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 
                                           'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 
                                           'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 
                                           'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month', 
                                           'Amount_invested_monthly', 'Monthly_Balance'])

In [21]:
encoded_cols = encoded_cols.set_index(standardize_cols.index)

for column_name in encoded_cols.columns:
    standardize_cols[column_name] = encoded_cols[column_name]

In [22]:
Y_train = Y_train.to_frame().set_index(standardize_cols.index)

standardize_cols['Credit_Score'] = Y_train

#### Export Data to Excel

In [23]:
datatoexcel = pd.ExcelWriter('Scaled.xlsx')
 
# write DataFrame to excel
standardize_cols.to_excel(datatoexcel)
 
# save the excel
datatoexcel.close()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [24]:
print(standardize_cols.isna().sum())



Annual_Income                                           0
Monthly_Inhand_Salary                                   0
Num_Bank_Accounts                                       0
Num_Credit_Card                                         0
Interest_Rate                                           0
Num_of_Loan                                           234
Delay_from_due_date                                     0
Num_of_Delayed_Payment                                  0
Changed_Credit_Limit                                   69
Num_Credit_Inquiries                                    0
Outstanding_Debt                                        0
Credit_Utilization_Ratio                                0
Credit_History_Age                                      0
Total_EMI_per_month                                     0
Amount_invested_monthly                                 0
Monthly_Balance                                         0
Payment_Behaviour_High_spent_Large_value_payments       0
Payment_Behavi

In [25]:
standardize_cols = standardize_cols.dropna()

In [26]:
print(standardize_cols.isna().sum())


Annual_Income                                         0
Monthly_Inhand_Salary                                 0
Num_Bank_Accounts                                     0
Num_Credit_Card                                       0
Interest_Rate                                         0
Num_of_Loan                                           0
Delay_from_due_date                                   0
Num_of_Delayed_Payment                                0
Changed_Credit_Limit                                  0
Num_Credit_Inquiries                                  0
Outstanding_Debt                                      0
Credit_Utilization_Ratio                              0
Credit_History_Age                                    0
Total_EMI_per_month                                   0
Amount_invested_monthly                               0
Monthly_Balance                                       0
Payment_Behaviour_High_spent_Large_value_payments     0
Payment_Behaviour_High_spent_Medium_value_paymen

In [27]:
datatoexcel = pd.ExcelWriter('Scaled.xlsx')
 
# write DataFrame to excel
standardize_cols.to_excel(datatoexcel)
 
# save the excel
datatoexcel.close()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


### Splitting df into Train/Test Set

In [ ]:
X = standardize_cols.iloc[:, 0:27]
y = standardize_cols.iloc[:, 28]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [29]:
X_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,...,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No
4147,-0.418463,-0.434466,-0.089530,0.848815,0.199423,0.031292,0.807075,-0.094326,0.326611,0.107135,...,1,0,0,0,0,0,0,1,0,1
2506,-1.033987,-1.440682,-0.073226,-0.106474,0.638217,0.204946,0.874408,-0.054681,-1.499849,0.556110,...,0,0,0,1,0,1,0,0,0,0
3289,-0.180161,-0.259120,-0.081378,-0.151254,-0.060579,0.204946,-1.414920,-0.059086,0.357279,0.336737,...,0,0,0,0,1,0,0,1,0,0
3580,0.173055,0.402388,-0.113984,-0.151254,-0.225359,0.031292,-0.808921,-0.067896,-0.125821,-0.066551,...,0,0,0,0,1,0,0,1,1,0
701,-0.014699,0.154542,-0.122136,-0.136327,0.141094,0.378601,-0.943588,-0.081111,0.109894,0.382422,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,0.177056,0.293065,-0.122136,-0.128864,-0.225359,0.480183,-0.337589,-0.111946,0.900022,0.454509,...,0,0,1,0,0,0,0,1,0,0
2941,0.558596,0.790011,13.230200,-0.151254,-0.139219,0.031292,-0.202923,0.786667,-0.042718,0.208735,...,0,0,1,0,0,0,0,1,0,1
967,0.884816,1.203006,-0.105833,-0.151254,-0.139219,0.204946,-1.078254,-0.089921,0.328908,0.208735,...,0,0,0,0,0,0,1,0,0,1
4229,0.557503,0.759379,-0.089530,-0.128864,0.638217,0.581764,0.470409,-0.045872,0.202162,0.382422,...,0,0,0,0,0,1,0,0,0,0


In [30]:
y_train

4147    Standard
2506    Standard
3289    Standard
3580    Standard
701     Standard
          ...   
3078    Standard
2941    Standard
967         Poor
4229        Poor
247         Good
Name: Credit_Score, Length: 3726, dtype: object

### Model

In [31]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
class_keys = ['Good', 'Poor', 'Standard']


In [32]:

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units = 32, activation = "relu"),
#     tf.keras.layers.Dense(units = 16, activation = "relu"),
#     tf.keras.layers.Dense(units = 3, activation = "softmax")
# ])

# model.compile(optimizer = 'adam', 
#               loss = 'sparse_categorical_crossentropy',
#               metrics = ['accuracy'])

# model.fit(X_train, y_train, epochs = 2000)

In [33]:
# model.evaluate(X_test, y_test)

### Model with Dropout

In [34]:
# model_with_dropout = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units = 32, activation = "relu"),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(units = 16, activation = "relu"),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(units = 8, activation = "relu"),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(units = 3, activation = "softmax")
# ])

# model_with_dropout.compile(optimizer = 'adam', 
#               loss = 'sparse_categorical_crossentropy',
#               metrics = ['accuracy'])

# model_with_dropout.fit(X_train, y_train, epochs = 2000)

In [35]:
# model_with_dropout.evaluate(X_test, y_test)

### Model with Regularization

In [36]:
# model_with_l2 = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units = 32, activation = "relu", kernel_regularizer=regularizers.l2(0.01)),
#     tf.keras.layers.Dense(units = 16, activation = "relu", kernel_regularizer=regularizers.l2(0.01)),
#     tf.keras.layers.Dense(units = 8, activation = "relu", kernel_regularizer=regularizers.l2(0.01)),
#     tf.keras.layers.Dense(units = 3, activation = "softmax")
# ])

# model_with_l2.compile(optimizer = 'adam', 
#               loss = 'sparse_categorical_crossentropy',
#               metrics = ['accuracy'])

# model_with_l2.fit(X_train, y_train, epochs = 2000)

In [37]:
# model_with_l2.evaluate(X_test, y_test)

### Model with Regularization and Dropout

In [38]:
model_with_both = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units = 256, activation = tf.keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=regularizers.l2(0.005)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 256, activation = tf.keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=regularizers.l2(0.005)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 256, activation = tf.keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=regularizers.l2(0.005)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 3, activation = "softmax")
])

/opt/anaconda3/envs/default/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [39]:
model_with_both.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [40]:
model_with_both.fit(X_train, y_train, epochs = 100)

Epoch 1/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5633 - loss: 3.2480
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6751 - loss: 1.8772
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7199 - loss: 1.2866
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7281 - loss: 0.9854
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7429 - loss: 0.8442
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7324 - loss: 0.7495
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7349 - loss: 0.6997
Epoch 8/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7510 - loss: 0.6830
Epoch 9/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7447 - loss: 0.6430
Epoch 10/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7460 - loss: 0.6375
Epoch 11/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7485 - loss: 0.6193
Epoch 12/100
117/117 ━━━━━━━━━━━━━━━━━━━━

In [41]:
model_with_both.evaluate(X_test, y_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.7246 - loss: 0.6120


[0.6129317879676819, 0.7242489457130432]

### Decision Tree

In [42]:
import tensorflow_decision_forests as tfdf

forest_model = tfdf.keras.RandomForestModel()

Use /var/folders/kz/3t7pyllj7w3fy7h055vlb0p00000gn/T/tmpq097u45w as temporary training directory


In [45]:
train_forest, test_forest = train_test_split(standardize_cols, test_size = 0.2, random_state = 3)

train_forest = tfdf.keras.pd_dataframe_to_tf_dataset(train_forest, label="Credit_Score")
test_forest = tfdf.keras.pd_dataframe_to_tf_dataset(test_forest, label="Credit_Score")

forest_model.fit(train_forest)

Reading training dataset...
Training dataset read in 0:00:01.916370. Found 3726 examples.
Training model...
Model trained in 0:00:00.702698
Compiling model...


[INFO 24-07-12 21:07:35.9140 +08 kernel.cc:1233] Loading model from path /var/folders/kz/3t7pyllj7w3fy7h055vlb0p00000gn/T/tmpq097u45w/model/ with prefix e787b4f4202d40c1
[INFO 24-07-12 21:07:36.0847 +08 decision_forest.cc:734] Model loaded with 300 root(s), 144088 node(s), and 28 input feature(s).
[INFO 24-07-12 21:07:36.0848 +08 abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 24-07-12 21:07:36.0848 +08 kernel.cc:1061] Use fast generic engine


Model compiled.


In [62]:
forest_model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (28):
	Amount_invested_monthly
	Annual_Income
	Changed_Credit_Limit
	Credit_History_Age
	Credit_Mix_Bad
	Credit_Mix_Good
	Credit_Mix_Standard
	Credit_Utilization_Ratio
	Delay_from_due_date
	Interest_Rate
	Monthly_Balance
	Monthly_Inhand_Salary
	Num_Bank_Accounts
	Num_Credit_Card
	Num_Credit_Inquiries
	Num_of_Delayed_Payment
	Num_of_Loan
	Outstanding_Debt
	Payment_Behaviour_High_spent_Large_value_payments
	Payment_Behaviour_High_spent_Medium_value_payments
	Payment_Behaviour_High_spent_Small_value_payments
	Payment_Behaviour_Low_spent_Large_value_payments
	Payment_Behaviour_Low_spent_Medium_valu

In [46]:
forest_model.compile(metrics=["accuracy"])
forest_model.evaluate(test_forest, return_dict=True)

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.8036


{'loss': 0.0, 'accuracy': 0.803648054599762}

In [47]:
preds = forest_model.predict(test_forest)

1/1 [==============================] - 0s 89ms/step
